Overlap Masks

In [ ]:

from PIL import Image
import numpy as np
import os
import re
import matplotlib.pyplot as plt

def maximum_intensity_projection(images):
    """ Apply maximum intensity projection to a list of images """
    # Ensure all images are in grayscale
    np_images = [np.array(img.convert('L')) for img in images]

    # Find the maximum intensity for each pixel
    max_image = np.maximum.reduce(np_images)

    # Convert back to PIL Image in 'L' mode
    return Image.fromarray(max_image, mode='L')

def colorize_mask(mask, color):
    """ Colorize the non-zero areas of a grayscale mask """
    # Convert mask to grayscale ('L') and then to numpy array
    mask_array = np.array(mask.convert('L'))

    # Create an RGBA image of the same size - initialized to transparent
    colored_mask = np.zeros((mask_array.shape[0], mask_array.shape[1], 4), dtype=np.uint8)

    # Apply color only to non-zero parts of the mask
    mask_area = mask_array > 0  # Identify non-zero areas (mask areas)
    colored_mask[mask_area] = list(color)  # Colorize only the mask area

    # Convert back to PIL Image in 'RGBA' mode
    return Image.fromarray(colored_mask, mode='RGBA')

def combine_masks(masks):
    """ Combine multiple RGBA masks """
    combined_mask = Image.new('RGBA', masks[0].size)
    for mask in masks:
        combined_mask = Image.alpha_composite(combined_mask, mask)
    return combined_mask

######CHANGE FROM HERE ON#####

# Define colors for each celltypes
colors = {
    'CD4': (0, 255, 0, 128),    # Green
    'CD8': (255, 0, 0, 128),    # Red
    'FoxP3': (0, 0, 255, 128),  # Blue
    'panCK': (255, 255, 0, 128),  # Yellow
    'KRT10': (128, 0, 128, 128)  # Purple (using RGB values for purple)
}

celltypes = ['CD4', 'CD8', 'FoxP3', 'panCK','KRT10']


# Directory where the images are stored
directory = '/Users/piastadler/Desktop/Stacking/AD1_020624/original_masks/'

# Initialize an empty set to store unique tiles
m_tiles = set()

# Iterate through all files in the directory
for filename in os.listdir(directory):
    # Check if the file is a .tif file and starts with one of the prefixes
    if filename.endswith('.tif') and any(filename.startswith(celltype) for celltype in celltypes):
        # Extract the tile part from the filename using regex
        # Assuming the pattern: prefix_p1_wA1_t1_tile_c5_z1_l1_o0.tif
        match = re.search(r'_p1_wA1_t1_(.+?)_c5_z1_l1_o0.tif', filename)
        if match:
            tile = match.group(1)
            m_tiles.add(tile)

# Convert the set to a sorted list
m_tiles = sorted(list(m_tiles))

print("Extracted tiles:", m_tiles)


# Other parts of the filename
suffix = '_p1_wA1_t1_{}_c5_z1_l1_o0.tif'  # The {} will be replaced by the m variation

#def remove_overlap(mask1, mask2):
  #  """ Remove overlapping areas from two masks """
    #overlap = np.logical_and(mask1, mask2)
   # return np.logical_and(mask1, np.logical_not(overlap)), np.logical_and(mask2, np.logical_not(overlap))

for m_tile in m_tiles:
    masks = {}
    for celltype in celltypes:
        filename = directory + celltype + suffix.format(m_tile)
        try:
            img = Image.open(filename).convert('L')
            masks[celltype] = np.array(img) > 0  # Create a binary mask
            print(f"Loaded image for {celltype} with {m_tile}: {filename}")
        except FileNotFoundError:
            print(f"File not found: {filename}")

    #if 'CD4' in masks and 'CD8' in masks:
        # Remove overlaps between CD4 and CD8
       # masks['CD4'], masks['CD8'] = remove_overlap(masks['CD4'], masks['CD8'])

    # Colorize and combine the masks
    colored_masks = []
    for celltype, mask in masks.items():
        colored_mask = colorize_mask(Image.fromarray(mask.astype(np.uint8) * 255), colors[celltype])
        colored_masks.append(colored_mask)

    if colored_masks:
        combined_image = combine_masks(colored_masks)

        # Display and save the combined image
        plt.imshow(combined_image)
        plt.axis('off')
        plt.title(f"Combined image for {m_tile}")
        plt.show()

        combined_save_path = directory + f"combinedfiltered_{m_tile}.pdf"
        combined_image.save(combined_save_path)
        print(f"Combined image saved to: {combined_save_path}")

Filter Masks as wanted, Louping through all tiles

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tifffile
from PIL import Image
from scipy.ndimage import label
import re

# Directory where the images are stored
directory = '/Users/piastadler/Desktop/Stacking/AD1_020624/original_masks/'

# Output directories for saving masks and visualizations
output_mask_directory = os.path.join(directory, 'output_masks')
output_viz_directory = os.path.join(directory, 'output_visualizations')
os.makedirs(output_mask_directory, exist_ok=True)
os.makedirs(output_viz_directory, exist_ok=True)

# Different Tiles an segmented celltypes
celltypes = ['CD4', 'CD8', 'FoxP3', "KRT10", "panCK"]

# Initialize an empty set to store unique tiles
m_tiles = set()

# Iterate through all files in the directory
for filename in os.listdir(directory):
    # Check if the file is a .tif file and starts with one of the prefixes
    if filename.endswith('.tif') and any(filename.startswith(celltype) for celltype in celltypes):
        # Extract the tile part from the filename using regex
        # Assuming the pattern: prefix_p1_wA1_t1_tile_c5_z1_l1_o0.tif
        match = re.search(r'_p1_wA1_t1_(.+?)_c5_z1_l1_o0.tif', filename)
        if match:
            tile = match.group(1)
            m_tiles.add(tile)

# Convert the set to a sorted list
m_tiles = sorted(list(m_tiles))

print("Extracted tiles:", m_tiles)

def generate_random_color():
    """Generate a random color with full opacity."""
    return np.random.randint(0, 256, 3).tolist() + [255]

def save_image(image_array, path):
    """Save an image array to a file."""
    Image.fromarray(image_array).save(path)

def colorize_mask(mask):
    """Create a color overlay for a mask where each cell is uniquely colored."""
    rgba_image = np.zeros((mask.shape[0], mask.shape[1], 4), dtype=np.uint8)
    
    # Find unique cell identifiers in the mask, ignoring the background (0)
    unique_cells = np.unique(mask[mask > 0])
    
    # Apply a random color to each cell
    for cell_id in unique_cells:
        rgba_image[mask == cell_id, :3] = generate_random_color()[:3]
        rgba_image[mask == cell_id, 3] = 255  # Full opacity
    
    return rgba_image
    
#####CD8#####

for m_tile in m_tiles:
    print(f"Processing tile {m_tile}")
    
    # Load and process the masks for the current variation
    masks = {}
    processed_masks = {}

    for celltype in celltypes:
        filename = f"{celltype}_p1_wA1_t1_{m_tile}_c5_z1_l1_o0.tif"
        input_path = os.path.join(directory, filename)
        try:
            # Use tifffile to directly read the cell masks with unique labels
            mask = tifffile.imread(input_path)
            masks[celltype] = mask
            processed_masks[celltype] = colorize_mask(mask)
        except FileNotFoundError:
            print(f"File not found: {input_path}")
        
    # Remove overlapping regions and track excluded cells
    excluded_cells = {}

    if 'CD8' in masks and 'CD4' in masks:
        overlap = (masks['CD8'] > 0) & (masks['CD4'] > 0)
        cd8_labels_to_remove = np.unique(masks['CD8'][overlap])
        cd4_labels_to_remove = np.unique(masks['CD4'][overlap])

        excluded_cells['CD8'] = np.isin(masks['CD8'], cd8_labels_to_remove) & (masks['CD8'] > 0)
        excluded_cells['CD4'] = np.isin(masks['CD4'], cd4_labels_to_remove) & (masks['CD4'] > 0)

        for label_cd8 in cd8_labels_to_remove:
            masks['CD8'][masks['CD8'] == label_cd8] = 0

        for label_cd4 in cd4_labels_to_remove:
            masks['CD4'][masks['CD4'] == label_cd4] = 0

    # Convert labeled masks back to binary and save CD8 masks
    if 'CD8' in masks:
        final_mask = masks['CD8'] > 0  # Convert to binary mask

        # Combine original and excluded masks for visualization
        combined_mask = np.zeros((final_mask.shape[0], final_mask.shape[1], 3), dtype=np.uint8)

        # Final cells in green
        combined_mask[final_mask] = [0, 255, 0]

        # Excluded cells in red
        combined_mask[excluded_cells['CD8']] = [255, 0, 0]

        # Show the combined visualization
        plt.imshow(combined_mask)
        plt.scatter([], [], color='green', label='CD8 Cells')
        plt.scatter([], [], color='red', label='Excluded Cells')
        legend = plt.legend(loc='upper right', fontsize='small', framealpha=0.3)  # Adjust font size and transparency
        plt.title(f"Final vs Excluded CD8 Cells for {m_tile}")
        plt.axis('off')
        plt.savefig(os.path.join(output_viz_directory, f"final_vs_excluded_CD8_{m_tile}.pdf"))  # Save the visualization
        plt.show()
        save_image(final_mask.astype(np.uint8) * 255, os.path.join(output_mask_directory, f"CD8_p1_wA1_t1_{m_tile}_c5_z1_l1_o0.tif"))
        excluded_cells.clear()  # Clearing the dictionary of excluded cells

######FoxP3#####

for m_tile in m_tiles:
    print(f"Processing tile {m_tile}")
    
    # Load and process the masks for the current variation
    masks = {}
    processed_masks = {}

    for celltype in celltypes:
        filename = f"{celltype}_p1_wA1_t1_{m_tile}_c5_z1_l1_o0.tif"
        input_path = os.path.join(directory, filename)
        try:
            # Use tifffile to directly read the cell masks with unique labels
            mask = tifffile.imread(input_path)
            masks[celltype] = mask
            processed_masks[celltype] = colorize_mask(mask)
        except FileNotFoundError:
            print(f"File not found: {input_path}")
        
    # Remove overlapping regions and track excluded cells
    excluded_cells = {}

  ## Exclude CD4+CD8+ Overlap ##
    if 'CD4' in masks and 'CD8' in masks:
        overlap_cd4_cd8 = (masks['CD4'] > 0) & (masks['CD8'] > 0)
        cd8_labels_to_remove = np.unique(masks['CD8'][overlap_cd4_cd8])
        cd4_labels_to_remove = np.unique(masks['CD4'][overlap_cd4_cd8])

# Create a mask for excluded CD4+CD8+ cells before setting their labels to 0
        excluded_cd4_cd8_mask = np.isin(masks['CD4'], cd4_labels_to_remove) & (masks['CD4'] > 0)

    for label in cd4_labels_to_remove:
        masks['CD4'][masks['CD4'] == label] = 0

## Identify FOXP3+ CD4+ Cells ##
    if 'CD4' in masks and 'FoxP3' in masks:
        overlap_cd4_foxp3 = (masks['CD4'] > 0) & (masks['FoxP3'] > 0)
        foxp3_cd4_labels = np.unique(masks['CD4'][overlap_cd4_foxp3])

    # Create a mask for FOXP3+ CD4+ cells, named 'FOXP3'
    masks['FOXP3'] = np.isin(masks['CD4'], foxp3_cd4_labels) & (masks['CD4'] > 0)

## Identify FOXP3- CD4+ Cells ##
    # Create a mask for FOXP3- CD4+ cells, keeping the name 'CD4'
    # This involves identifying CD4 labels not in foxp3_cd4_labels
    foxp3_neg_cd4_labels = np.setdiff1d(np.unique(masks['CD4']), foxp3_cd4_labels)
    masks['CD4'] = np.isin(masks['CD4'], foxp3_neg_cd4_labels) & (masks['CD4'] > 0)

# Visualization and saving logic can be added here for both 'FOXP3' and 'CD4' masks

# Save the FOXP3 positive CD4 mask
    if 'FOXP3' in masks:
        final_mask_foxp3 = masks['FOXP3'] > 0
        save_image(final_mask_foxp3.astype(np.uint8) * 255, os.path.join(output_mask_directory, f"FOXP3_p1_wA1_t1_{m_tile}_c5_z1_l1_o0.tif"))

# Save the FOXP3 negative CD4 mask
    if 'CD4' in masks:
        final_mask_cd4_foxp3_neg = masks['CD4'] > 0
        save_image(final_mask_cd4_foxp3_neg.astype(np.uint8) * 255, os.path.join(output_mask_directory, f"CD4_p1_wA1_t1_{m_tile}_c5_z1_l1_o0.tif"))
    
    # Create a combined visualization
        combined_mask = np.zeros((masks['CD4'].shape[0], masks['CD4'].shape[1], 3), dtype=np.uint8)

    # FOXP3- CD4+ cells in green
        combined_mask[final_mask_cd4_foxp3_neg] = [0, 255, 0]

    # FOXP3+ CD4+ cells in blue
        combined_mask[final_mask_foxp3] = [0, 0, 255]

    # Excluded CD4+CD8+ overlap cells in red
        combined_mask[excluded_cd4_cd8_mask] = [255, 0, 0]

    # Show the combined visualization
        plt.imshow(combined_mask)
        plt.scatter([], [], color='green', label='FOXP3- CD4+ Cells')
        plt.scatter([], [], color='blue', label='FOXP3+ CD4+ Cells')
        plt.scatter([], [], color='red', label='Excluded Cells')
        legend = plt.legend(loc='upper right', fontsize='small', framealpha=0.3)  # Adjust font size and transparency
        plt.title(f"FOXP3+ CD4+ vs FOXP3- CD4+ Cells for {m_tile}")
        plt.axis('off')
        plt.savefig(os.path.join(output_viz_directory, f"FOXP3_CD4_combined_{m_tile}.pdf"))  # Save the visualization
        plt.show()
        excluded_cells.clear()  # Clearing the dictionary of excluded cells

######suprabasal/basal######

for m_tile in m_tiles:
    
    # Load and process the masks for the current variation
    masks = {}
    processed_masks = {}

    for celltype in celltypes:
        filename = f"{celltype}_p1_wA1_t1_{m_tile}_c5_z1_l1_o0.tif"
        input_path = os.path.join(directory, filename)
        try:
            # Use tifffile to directly read the cell masks with unique labels
            mask = tifffile.imread(input_path)
            masks[celltype] = mask
            processed_masks[celltype] = colorize_mask(mask)
        except FileNotFoundError:
            print(f"File not found: {input_path}")
    
    # Remove overlapping regions and track excluded cells
    excluded_cells = {}
    
    # List of cell types to check for overlap with panCK
    cell_types_to_exclude = ['CD8', 'CD4']

    if 'panCK' in masks:  # Check if 'panCK' key exists
        # Initialize a mask to accumulate all excluded cells
        excluded_mask = np.zeros_like(masks['panCK'], dtype=bool)

        for cell_type in cell_types_to_exclude:
            if cell_type in masks:
                # Find overlaps between panCK and the current cell type
                overlap = (masks['panCK'] > 0) & (masks[cell_type] > 0)

                # Get unique labels of panCK and the other cell type in the overlap
                panCK_labels_to_remove = np.unique(masks['panCK'][overlap])

                # Update the excluded mask
                excluded_mask |= np.isin(masks['panCK'], panCK_labels_to_remove) & (masks['panCK'] > 0)

                # Remove the identified overlapping panCK shapes
                for label in panCK_labels_to_remove:
                    masks['panCK'][masks['panCK'] == label] = 0

    ## Identify KRT10+ panCK+ Cells ##
    if 'panCK' in masks and 'KRT10' in masks:
        overlap_panCK_KRT10 = (masks['panCK'] > 0) & (masks['KRT10'] > 0)
        KRT10_panCK_labels = np.unique(masks['panCK'][overlap_panCK_KRT10])

        # Create a mask for FOXP3+ CD4+ cells, named 'FOXP3'
        masks['KRT10_panCK_pos'] = np.isin(masks['panCK'], KRT10_panCK_labels) & (masks['panCK'] > 0)

    ## Identify KRT10- panCK+ Cells ##
        KRT10neg_panCK_labels = np.setdiff1d(np.unique(masks['panCK']), KRT10_panCK_labels)
        masks['KRT10_panCK_neg'] = np.isin(masks['panCK'], KRT10neg_panCK_labels) & (masks['panCK'] > 0)

    # Visualization and saving logic can be added here for both 'FOXP3' and 'CD4' masks

    # Save the KRT10+ panCK+ Cells
    if 'KRT10_panCK_pos' in masks:
        final_mask_KRT10_pos = masks['KRT10_panCK_pos'] > 0  # Ensure the mask is binary
        save_image(final_mask_KRT10_pos.astype(np.uint8) * 255, os.path.join(output_mask_directory, f"KRT10_p1_wA1_t1_{m_tile}_c5_z1_l1_o0.tif"))

    # Save the KRT10- panCK+ Cells
    if 'KRT10_panCK_neg' in masks:
        final_mask_KRT10_neg = masks['KRT10_panCK_neg'] > 0  # Ensure the mask is binary
        save_image(final_mask_KRT10_neg.astype(np.uint8) * 255, os.path.join(output_mask_directory, f"panCK_p1_wA1_t1_{m_tile}_c5_z1_l1_o0.tif"))

    # Create a combined visualization
    combined_mask = np.zeros((masks['panCK'].shape[0], masks['panCK'].shape[1], 3), dtype=np.uint8)

    # KRT10- panCK+ cells in green
    combined_mask[masks['KRT10_panCK_neg']] = [0, 255, 0]

    # KRT10+ panCK+ cells in blue
    combined_mask[masks['KRT10_panCK_pos']] = [0, 0, 255]

    # Excluded cells (both CD4+ and CD8+ overlaps with panCK) in red
    combined_mask[excluded_mask] = [255, 0, 0]

    # Show the combined visualization
    plt.imshow(combined_mask)
    plt.scatter([], [], color='green', label='KRT10- panCK+ Cells')
    plt.scatter([], [], color='blue', label='KRT10+ panCK+ Cells')
    plt.scatter([], [], color='red', label='Excluded Cells')
    legend = plt.legend(loc='upper right', fontsize='small', framealpha=0.3)  # Adjust font size and transparency
    plt.title(f"KRT10+ panCK+ vs KRT10- panCK+ Cells for {m_tile}")
    plt.axis('off')
    plt.savefig(os.path.join(output_viz_directory, f"panCK_KRT10_combined_{m_tile}.pdf"))  # Save the visualization
    plt.show()


Filter for size

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import label
import os
import re
import tifffile 

# Different Tiles and segmented celltypes
celltypes = ['CD4', 'CD8', 'FoxP3', "KRT10", "panCK"]

# Directory where the images are stored
directory = '/Users/piastadler/Desktop/Stacking/AD1_020624/output_masks/'

# Initialize an empty set to store unique tiles
m_tiles = set()

# Iterate through all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.tif') and any(filename.startswith(celltype) for celltype in celltypes):
        match = re.search(r'_p1_wA1_t1_(.+?)_c5_z1_l1_o0.tif', filename)
        if match:
            tile = match.group(1)
            m_tiles.add(tile)

m_tiles = sorted(list(m_tiles))
print("Extracted tiles:", m_tiles)

output_directory = os.path.join(directory, 'output_reduced')
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

shape_sizes = {celltype: [] for celltype in celltypes}

# First Loop: Collect sizes without applying filtering
for m_tile in m_tiles:
    for celltype in celltypes:
        filename = f"{celltype}_p1_wA1_t1_{m_tile}_c5_z1_l1_o0.tif"
        input_path = os.path.join(directory, filename)

        try:
            mask = tifffile.imread(input_path)
            labeled_array, num_features = label(mask > 0)

            for i in range(1, num_features + 1):
                size = np.sum(labeled_array == i)
                shape_sizes[celltype].append(size)

        except FileNotFoundError:
            print(f"File not found: {input_path}")

# Compute cutoff values based on the collected sizes
cutoffs = {}
for celltype, sizes in shape_sizes.items():
    if sizes:
        cut_off_left = np.exp(np.percentile(np.log(sizes), 5))
        cut_off_right = np.exp(np.percentile(np.log(sizes), 95))
        cutoffs[celltype] = (cut_off_left, cut_off_right)

# Second Loop: Apply size filtering and update masks
for m_tile in m_tiles:
    for celltype in celltypes:
        cut_off_left, cut_off_right = cutoffs[celltype]  # Retrieve the cutoffs for the current cell type

        filename = f"{celltype}_p1_wA1_t1_{m_tile}_c5_z1_l1_o0.tif"
        input_path = os.path.join(directory, filename)

        try:
            mask = tifffile.imread(input_path)
            labeled_array, num_features = label(mask > 0)
            new_mask = np.zeros_like(mask)

            for i in range(1, num_features + 1):
                size = np.sum(labeled_array == i)
                if cut_off_left <= size <= cut_off_right:
                    new_mask[labeled_array == i] = i

            # Save the new mask
            new_filename = f"{celltype}_p1_wA1_t1_{m_tile}_c5_z1_l1_o0.tif"
            output_path = os.path.join(output_directory, new_filename)
            tifffile.imwrite(output_path, new_mask)

            print(f"Filtered mask saved: {output_path}")

        except FileNotFoundError:
            print(f"File not found: {input_path}")

# Plotting log-transformed size distributions
for celltype, sizes in shape_sizes.items():
    if sizes:
        plt.figure(figsize=(10, 6))
        log_sizes = np.log(sizes)  # Apply log transformation

        n_shapes_before = len(sizes)  # Total number of shapes before filtering

        # Calculate median and cutoffs for log-transformed sizes
        median_log_size = np.median(log_sizes)
        log_cut_off_left, log_cut_off_right = np.log(cutoffs[celltype])

        # Filter the log-transformed sizes based on the log-transformed cutoffs
        filtered_log_sizes = [size for size in log_sizes if log_cut_off_left <= size <= log_cut_off_right]
        n_shapes_after = len(filtered_log_sizes)  # Total number of shapes after filtering

        # Plot the histogram of log-transformed, filtered sizes
        plt.hist(log_sizes, bins=100, alpha=0.7, label=celltype)
        plt.xlabel("Log of Shape Size (pixels)")
        plt.ylabel("Frequency")
        plt.title(f"{celltype} Log Shape Size Distribution\nTotal shapes before: {n_shapes_before}, Total shapes after: {n_shapes_after}")

        # Add vertical lines for median and cutoffs
        plt.axvline(median_log_size, color='red', linestyle='dashed', linewidth=1, label='Median')
        plt.axvline(log_cut_off_left, color='green', linestyle='dashed', linewidth=1, label='Cut Off Left')
        plt.axvline(log_cut_off_right, color='blue', linestyle='dashed', linewidth=1, label='Cut Off Right')

        plt.legend()

        output_filename = f"Log_Transformed_Distribution_{celltype}.pdf"
        output_path = os.path.join(output_directory, output_filename)
        plt.savefig(output_path, format='pdf', dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Saved log-transformed size distribution plot for {celltype} to {output_path}")

Rename to import in BIAS

In [ ]:
import os
import shutil
import tifffile

# Directory where the original images are stored
source_directory = '/Users/piastadler/Desktop/Stacking/AD1_020624/output_reduced/'  # Update this path to your source directory

# Directory where the organized images will be stored
target_directory = '/Users/piastadler/Desktop/Stacking/AD1_020624/output_reduced/Organized/'  # Update this path to your target directory

# Check if the target directory exists, if not, create it
if not os.path.exists(target_directory):
    os.makedirs(target_directory)

# Loop through all files in the source directory
for filename in os.listdir(source_directory):
    # Check if the file is a .tif file
    if filename.endswith('.tif'):
        # Extract the cell type from the filename (assuming the cell type is at the beginning followed by an underscore)
        celltype = filename.split('_')[0]

        # Create a directory for the cell type if it doesn't already exist
        celltype_directory = os.path.join(target_directory, celltype)
        if not os.path.exists(celltype_directory):
            os.makedirs(celltype_directory)

        # Define the new filename without the cell type prefix
        new_filename = '_'.join(filename.split('_')[1:])

        # Full path for the source and target files
        source_file = os.path.join(source_directory, filename)
        target_file = os.path.join(celltype_directory, new_filename)

        # Optionally, you can use tifffile to read and then save the image to ensure it's correctly processed
        # This step can be skipped if a simple file move is sufficient
        image = tifffile.imread(source_file)
        tifffile.imwrite(target_file, image)

        # Alternatively, for a simple move without reading and writing the image, uncomment the next line and comment out the tifffile lines above
        # shutil.move(source_file, target_file)

        print(f"Moved {filename} to {celltype}/{new_filename}")

print("Organizing completed.")